<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/PostAnnotation/Notebooks/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers==4.28.0
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 11.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-dvt9unow
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-dvt9unow
  Resolved https://github.com/huggingface/accelerate to commit 5d558f21e27744491219002a375e158eee987e0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-

In [2]:
import pandas as pd
import numpy as np
import datasets

# Hugging Face

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Gdrive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/"

Mounted at /content/gdrive


# Dataset


In [5]:
train_ds = datasets.load_dataset("csv", data_files={"train": f"{dir}dataset/final/combined_train.csv"})
val_ds = datasets.load_dataset("csv", data_files={"validation": f"{dir}dataset/final/combined_validation.csv"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

## Prep

* https://huggingface.co/docs/transformers/tasks/sequence_classification
* https://huggingface.co/docs/transformers/model_doc/electra
* https://huggingface.co/google/electra-base-discriminator





In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator",
                                          do_lower_case=False)


In [7]:
#  pytorch
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
# Non k-fold
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,
    max_length=128)

train_tokenized = train_ds.map(preprocess_function, batched=True)
val_tokenized = val_ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/46262 [00:00<?, ? examples/s]

Map:   0%|          | 0/11566 [00:00<?, ? examples/s]

In [9]:
import evaluate

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)

    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(accuracy_metric.compute(predictions=preds, references = labels))
    return results

In [10]:
id2label, label2id = {},{}
for x in range(15):
    id2label[x] = x
        # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id[x] = x
        # {"NEGATIVE": 0, "POSITIVE": 1}

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer



model = AutoModelForSequenceClassification.from_pretrained("google/electra-base-discriminator",
                                                           num_labels=15,
                                                           id2label=id2label,
                                                           label2id=label2id
)

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d

# Training

In [12]:
#Non-KFold
training_args = TrainingArguments(
    output_dir="Google/Electra-Thesis-NonKFold",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,                    #10
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_tokenized['train'], #mfc_tokenized["train"]
    eval_dataset=val_tokenized['validation'],  #mfc_tokenized["test"]
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/jmLuis/Electra-Thesis-NonKFold into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Recall,Accuracy
1,1.653100,1.368801,0.585077,0.585077,0.585077
2,1.240400,1.267412,0.623033,0.623033,0.623033
3,1.054400,1.238934,0.632198,0.632198,0.632198
4,0.918900,1.229330,0.637558,0.637558,0.637558
5,0.803400,1.312989,0.638423,0.638423,0.638423
6,0.735100,1.346337,0.639028,0.639028,0.639028
7,0.648100,1.385819,0.638509,0.638509,0.638509
8,0.588500,1.437867,0.637213,0.637213,0.637213


Epoch,Training Loss,Validation Loss,F1,Recall,Accuracy
1,1.653100,1.368801,0.585077,0.585077,0.585077
2,1.240400,1.267412,0.623033,0.623033,0.623033
3,1.054400,1.238934,0.632198,0.632198,0.632198
4,0.918900,1.229330,0.637558,0.637558,0.637558
5,0.803400,1.312989,0.638423,0.638423,0.638423
6,0.735100,1.346337,0.639028,0.639028,0.639028
7,0.648100,1.385819,0.638509,0.638509,0.638509
8,0.588500,1.437867,0.637213,0.637213,0.637213
9,0.553300,1.479188,0.634446,0.634446,0.634446
10,0.528500,1.491367,0.639028,0.639028,0.639028


TrainOutput(global_step=14460, training_loss=0.8815881784526144, metrics={'train_runtime': 7396.1937, 'train_samples_per_second': 62.548, 'train_steps_per_second': 1.955, 'total_flos': 2.02891809749121e+16, 'train_loss': 0.8815881784526144, 'epoch': 10.0})

In [13]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Sep13_03-20-29_6f75f0fa1da3/events.out.tfevents.1694575239.6f75f0fa1da3.445.0:   0%|         …

To https://huggingface.co/jmLuis/Electra-Thesis-NonKFold
   aaaeb67..96f6a8d  main -> main

   aaaeb67..96f6a8d  main -> main

To https://huggingface.co/jmLuis/Electra-Thesis-NonKFold
   96f6a8d..74f9194  main -> main

   96f6a8d..74f9194  main -> main



'https://huggingface.co/jmLuis/Electra-Thesis-NonKFold/commit/96f6a8d7783eb24d450ccf441c5a7e12f462e997'

# Inference

In [16]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer


In [ ]:
model_name = 'Google/Electra-Thesis-NonKFold'

In [ ]:
# text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = 'Immigrants without HOPE need help entering college'
# text = 'Nan'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(f"jmLuis/{model_name}")
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForSequenceClassification.from_pretrained(f"jmLuis/{model_name}")

with torch.no_grad():
    logits = model(**inputs).logits


predicted_class_id = logits.argmax().item()
out = model.config.id2label[predicted_class_id]
out+1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(f"jmLuis/{model_name}")

model = AutoModelForSequenceClassification.from_pretrained(f"jmLuis/{model_name}")


def infer_electra(row):
    inputs = tokenizer(str(row['Text']), return_tensors="pt",  truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    out = model.config.id2label[predicted_class_id]
    return int(out)+1  #adding onw because original label is not zero index, however training is.

def doPrediction(df, filename, last, save=0):
    for i, row in df.iterrows():
        df.at[i,'Annotate'] = infer_electra(row)
        if(i%100==0):
            print(i)
        if (i%2000==0):
            print('saved', save)
            fn = filename+'-save-'+str(save)+'.csv'
            print(filename)
            df.to_csv(fn, index=False)
            save+=1
    df.to_csv(last, index=False)